In [1]:
import os
import h5py
from nltk.util import usage
import numpy as np
import datetime
import time
import warnings

from featureExtraction.tf.tf import *
from featureExtraction.tfidf.tfidf import *
from featureExtraction.fasttext.fasttext import *
from featureExtraction.lexicon.lexicon import *
from featureExtraction.orthography.orthography import *
from featureExtraction.pos.postag import *
from featureExtraction.tf.tf import *
from featureExtraction.word2vec.word2vec import *
from featureExtraction.ngram.ngram import *

from featureSelection.kFold import *

from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, cross_val_score

In [2]:
templated = [x for x in os.listdir("./Dataset/templated")]
templated.sort()
clean = [x for x in os.listdir("./Dataset/clean")]
clean.sort()

DATA_TEMPLATED = ["./Dataset/templated/" + x for x in os.listdir("./Dataset/templated")]
DATA_TEMPLATED.sort()
len_templated = len(DATA_TEMPLATED)
DATA_TEMPLATED1 = [DATA_TEMPLATED[0]]
DATA_CLEAN = ["./Dataset/clean/" + x for x in os.listdir("./Dataset/clean")]
DATA_CLEAN.sort()
len_clean = len(DATA_CLEAN)

In [3]:
warnings.filterwarnings("ignore")
score_list = []
count = 1
fail = 0
# for i in DATA_CLEAN[10:]:
for i in DATA_CLEAN:
    print("Running {}/{}".format(count, len_clean), end=" ")
    try:
        # timer mulai
        # result_fe, label, feat_name = run_lexiconVania_tweet(i)
        # result_fe, label, feat_name = run_lexiconInset_tweet(i)
        result_fe, label, feat_name = run_lexiconCombined_tweet(i)

        # create dataset
        # X, y = make_classification(n_samples=len(result_fe), n_features=len(result_fe[0]), random_state=1)
        # prepare the cross-validation procedure

        cv = KFold(n_splits=20, random_state=1, shuffle=True)
        x_train, x_test, y_train, y_test = train_test_split(result_fe, label)
                
        logRes = LogisticRegression(random_state=1, max_iter=1000)
        logRes.fit(x_train, y_train)
        predictions = logRes.predict(x_test)
        
        scores = cross_val_score(logRes, result_fe, label, scoring='f1_micro', cv=cv, n_jobs=-1)
        print('F1-score: %.3f (%.3f)' % (mean(scores), std(scores)))
        score_list.append(mean(scores))
        count += 1
        # timer selesai
    except Exception as e:
        count += 1
        fail += 1
        print(i, ": ", e)
        pass

print((sum(score_list)/len(score_list)))
print("Failed: {}".format(fail))

Running 1/20 F1-score: 0.677 (0.113)
Running 2/20 F1-score: 0.570 (0.109)
Running 3/20 F1-score: 0.632 (0.040)
Running 4/20 F1-score: 0.806 (0.076)
Running 5/20 F1-score: 0.736 (0.035)
Running 6/20 F1-score: 0.882 (0.018)
Running 7/20 F1-score: 0.796 (0.068)
Running 8/20 F1-score: 0.820 (0.084)
Running 9/20 F1-score: 0.771 (0.141)
Running 10/20 F1-score: 0.828 (0.123)
Running 11/20 F1-score: 0.665 (0.074)
Running 12/20 F1-score: 0.725 (0.151)
Running 13/20 F1-score: 0.644 (0.094)
Running 14/20 F1-score: 0.673 (0.150)
Running 15/20 F1-score: 0.593 (0.052)
Running 16/20 F1-score: 0.605 (0.029)
Running 17/20 F1-score: 0.493 (0.051)
Running 18/20 F1-score: 0.493 (0.051)
Running 19/20 F1-score: 0.492 (0.049)
Running 20/20 F1-score: 0.644 (0.105)
0.6772835603982837
Failed: 0
